In [8]:
%pip install -qU pypdf langchain_community

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.document_loaders import PyPDFLoader
file_path = "SLA_AnnualReport.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

132


In [10]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

SRILANKAN AIRLINES | ANNUAL REPORT 2023/24
www.srilankan.com
Airline Centre, 
Bandaranaike Internati
{'source': 'SLA_AnnualReport.pdf', 'page': 0}


In [11]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [12]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

 ········


In [13]:
%pip install langchain_openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What initiatives has SriLankan Airlines implemented to promote Sri Lanka as a tourist destination?"})

results

{'input': 'What initiatives has SriLankan Airlines implemented to promote Sri Lanka as a tourist destination?',
 'context': [Document(id='92cff695-cac1-4d5e-becb-96517f403d8c', metadata={'source': 'SLA_AnnualReport.pdf', 'page': 33}, page_content='franchisees of the Airline’s tour operating \narm, SriLankan Holidays. Through \ncarefully curated tours and accompanied \nby members of the Airline’s worldwide \nsales team, the participants were able \nto visit some of Sri Lanka’s most iconic \nlandmarks.\nSports and Culture Tourism\nOver the years, SriLankan Airlines has \nbeen a driving force in promoting the \nisland for adventure and sports tourism \nthrough an assortment of events in the \nspheres of surfing‚ cycling‚ mountain \nbiking‚ golfing‚ and running. The Airline \nexecuted a range of promotional activities \nfocusing on the Asia Cup and the historic \nKandy Esala Perahera; two remarkable \nevents that were simultaneously \nunderway. Through each of these events, \nthe Company s

In [16]:
print(results["context"][0].page_content)

franchisees of the Airline’s tour operating 
arm, SriLankan Holidays. Through 
carefully curated tours and accompanied 
by members of the Airline’s worldwide 
sales team, the participants were able 
to visit some of Sri Lanka’s most iconic 
landmarks.
Sports and Culture Tourism
Over the years, SriLankan Airlines has 
been a driving force in promoting the 
island for adventure and sports tourism 
through an assortment of events in the 
spheres of surfing‚ cycling‚ mountain 
biking‚ golfing‚ and running. The Airline 
executed a range of promotional activities 
focusing on the Asia Cup and the historic 
Kandy Esala Perahera; two remarkable 
events that were simultaneously 
underway. Through each of these events, 
the Company successfully showcased the 
Country’s exceptional beauty and rich 
culture, inspiring tens of thousands of 
tourists to visit. The Airline also returned 
to its two signature events in Australia, 
the iconic Melbourne Marathon and the


In [17]:
print(results["context"][0].metadata)

{'source': 'SLA_AnnualReport.pdf', 'page': 33}
